# Joke Recommender
Brendan Liu, Elaine Sieng, Josh Kim, Yerem Istanboulian

In [10]:
import pandas as pd
import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from IPython.display import Image

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
from sklearn.feature_extraction import text
from tqdm import tqdm, tqdm_notebook

import json

## Abstract

Recommender systems are techniques that attempt to suggest relevant items to users from very large and complex datasets. The dataset used includes over 4.1 million continuous ratings (-10.00 to +10.00) of 100 jokes from 73,421 users collected by Ken Goldberg and Theresa Roeder and Dhruv Gupta and Chris Perkins at UC Berkeley. Specifically, our joke recommender system will implement collaborative filtering methods to filter items that a user may like based on similar users. Using model based approaches uncover user-joke interactions which are then used to make further recommendations. This report will discuss specific matrix decomposition methods, component analysis and use a K-nearest neighbors approach to compare to a classic matrix approach. 

## Introduction
Recommender systems are quintessential in the connected world. Many different businesses often face the arduous task of recommending items to users without knowing the preferences of said users. Starting in the 1990’s research in recommender systems have grown significantly and have developed great efficiency in predicting the preferences of users.1 According to Zhu et al., a recommender system is based on three important pieces of data: users, items (e.g., movies, jokes, news), and user-item usage history (e.g., ratings). The goal is to attempt to understand the relationship between the users, items and user-item history, and to use that understanding to predict future user-item interactions.2 To tackle these tasks, over the last three decades, researchers have developed methods like non-negative matrix factorization (NMF), K-nearest neighbors (KNN), neural autoencoders, and many more. These models can be further grouped into several categories, among which a very important branch is collaborative filtering (CF).

CF can be implemented using many different approaches. The central data source of a CF method is the user-review matrix, with each entry corresponding to a numerical rating of an item within a finite itemset. These matrices tend to be extremely sparse, with many users and items. Our application of matrix methods attempts to find and extract a low ranking representation of the original user-ratings matrix.
 
In our work, we explore three different matrix decomposition methods for topic discovery and user clustering; and, we also utilize a KNN approach to recommending jokes to users.


## Data

Reading in data:

In [11]:
input_path = '../joke_recommender/data/'
df = pd.read_csv(input_path + 'joke_dataframe.csv')
df = df.drop(['Unnamed: 0', 'JokeId'], axis = 1)
joke_frame = pd.read_csv(input_path + 'JokeText.csv')
jokes = joke_frame.JokeText

Let's look at the head of the user-ratings matrix:

In [12]:
df.head(10)

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,-7.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,-5.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,-3.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,-1.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,4.03,...,3.64,4.32,6.99,-9.66,-8.40,-0.63,9.51,-7.67,-1.60,8.30
5,1.75,-0.78,1.26,6.65,8.25,-8.11,-6.75,2.14,0.34,1.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.76,1.60,-5.39,-7.52,4.08,4.42,-0.15,-0.24,-3.01,5.19,...,4.56,6.21,6.99,5.15,4.37,-3.64,-4.95,NaN,-5.29,-1.36
7,3.30,1.07,1.50,7.28,2.52,2.72,-5.87,8.06,-6.65,-0.29,...,0.87,-6.36,6.70,-9.37,-3.74,-7.23,8.59,-8.79,-3.69,0.29
8,-2.57,-8.69,-8.40,-5.15,-9.66,9.08,-3.54,2.82,-3.40,-3.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-1.41,-4.66,4.37,-7.14,2.48,9.13,-5.19,7.52,1.36,8.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Results and Discussion
**KMeans Clustering of Text - Topic Discovery**

To make the joke text machine readable, we used the TF-IDF vectorization technique. After vectorizing and normalizing the term-matrix, we used NMF to decompose this matrix instead of truncated singlular value decomposition (tSVD). This is because there is no interpretability of the negative values in tSVD. NMF ensures that the resulting decomposed term frequency matrix has interpretable positive values. NMF decomposes the matrix according to this algorithm:

$$ \min_{W,H} \| X - WH \|_F\\
\text{ subject to } W\geq 0,\ H\geq 0, $$  
    
where $W$ is ${p\times r}$ matrix and $H$ is ${r\times n}$ matrix. This also ensures only positive values in the resulting matrices.
    
After implementing NMF, we again passed the decomposed W column into the K-means clustering algorithm. We plot the score versus the number of clusters to choose the optimal number of clusters required.
<img src = "./images/elbow_method_joke_text.png">
The plotted elbow method is relatively smooth and seems parabolic. We determined that after 20 clusters, the lowered score was negligible. We plotted the first two principal components with the data points colored according to their respective cluster assignment.
<img src = "./images/Kmeans_Clustering_on_Text_Data.png">
We extracted the top words in each cluster and created a wordcloud of each one. A wordcloud is a visualization in which the highest frequency words of each cluster is plotted in order of descending frequency and text size. Some clusters have nonsensical clustering of words
<img src="images/Cluster19.png">
However, others have a clear 'theme' or 'pattern'. For example:
<img src="images/Cluster3.png">
Is clearly about engineer jokes.
<img src="images/Cluster4.png">
This one is clearly 'screw in lightbulb' jokes.

**PCA + User Clustering**

Our data is a high dimensional, sparse matrix with values ranging on a continuous scale between -10.0 and 10.0. In order to visualize our data, we performed principal component analysis (PCA), a statistical procedure that finds orthonormal transformations of variables called principal components. We performed an associated matrix factorization technique known as truncated singular value decomposition (tSVD) to decrease computation time and complexity. By using SVD, we cut our computation time in half. 
Upon computing tSVD on our user-ratings matrix, we plotted the cumulative variance explained as a function of total components.
<img src="images/PCA_users.png">
By inspection, 28 principal components were needed to describe 85% of all variability in the original matrix. A strong elbow curve indicated that each new component added positively increase the total variance explained, and is expected using tSVD. We chose to proceed with only the first 28 principal components for use in clustering. By plotting the first two components, we can see 3-4 different clusters by visual inspection.
<img src="images/tSVD_on_User_rat_mat.png">
tSVD is an appropriate method to use for clustering methods even though the matrices may have negative values. These negative values do not take away from the interpretability of our results.

Our next challenge was discerning different topics of jokes present in our body of joke text. From our decomposed matrix, a k-means algorithm was used to find appropriate clusters. We experimented with different number of clusters, and plotted the score of each cluster fit to find the optimal k-value. The elbow method allows us to see the k-value with the highest in between cluster separation and lowest inter-cluster variation.
<img src="./images/elbow_method_kmeans_users.png">
Past 6 clusters, the reduction is score is not as substantial, so we arrived at the choice of k = 6. By plotting the first two principal components and coloring in the data points according to the clusters assigned, we can see distinct groups of users.
<img src = "./images/Kmeans_Clustering_on_User_Data.png">
The grey points represents the center of each cluster, and all data points are colored according to their assigned cluster. Extracting the 5 highest rated jokes from each cluster, we get these joke indices:

$$
\begin{matrix}
& Rank1 & Rank2 & Rank3 & Rank4 & Rank5 \\
Cluster\ 1& 0 & 7 & 6 & 3 & 8\\
Cluster\ 2 & 0 & 2 & 3 & 5 & 7\\
Cluster\ 3 & 0 & 1 & 2 & 6 & 62 \\
Cluster\ 4 & 0 & 1 & 3 & 4 & 13 \\
Cluster\ 5 & 0 & 4 & 90 & 18 & 64\\
Cluster\ 6 & 0 & 3 & 15 & 5 & 10
\end{matrix}$$
This is very  interesting, because each cluster rated joke 0 highly.

**K-Nearest Neighbor Recommendations**

**Matrix Factorization Joke Recommendation**
<img src = "./images/residual_opt.png">
<img src = "./images/MF_Movie_heatmap.png">

## Experimental Considerations

## Future Work

## Conclusion